In [1]:
import os
import glob
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter
import yaml
from typing import Optional, Any, List
import time, datetime
from dotenv import load_dotenv
import os
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from langchain_community.vectorstores.faiss import FAISS
from openai import AzureOpenAI
from langchain.schema import Document
import pandas as pd

In [2]:
def find_jp_md_files(root_dir):
    jp_md_files = []
    for dir_path, _, files in os.walk(root_dir):
        for file in files:
            if file.endswith('.jp.md'):
                full_path = os.path.join(dir_path, file)
                jp_md_files.append(full_path)
    return jp_md_files

def preprocess_metadata(metadata_str):
    return metadata_str.replace("\t", " ")

def extract_metadata(metadata_str):
    metadata_str = preprocess_metadata(metadata_str)
    metadata = yaml.safe_load(metadata_str)
    return {
        "title":metadata.get("title", "").strip(""),
        "description": metadata.get("meta", {}).get("description", "").strip(""),
        "keywords": ",".join([k.strip("") for k in metadata.get("keywords", [])])
    }

def extract_path(file_path):
    # Extract the relevant path from the file path
    parts = file_path.split("content/")
    if len(parts) > 1:
        relevant_path = parts[1]
        relevant_path = os.path.dirname(relevant_path)
        return relevant_path
    return ""

def process_md_files(jp_md_files: List[str]) -> List[Document]:
    # Read all markdown files and extract metadata, content
    all_chunks = []
    for file_path in jp_md_files:
        with open(file_path, 'r', encoding='utf-8') as f:
            markdown_content = f.read()
        parts = markdown_content.split('---', 2)
        if len(parts) < 3:
            print(f"Skipping file {file_path}: Invalid format")
            continue

        # Extract metadata
        try:
            metadata_str = parts[1].strip("\n")
            extract_meta_dict = extract_metadata(metadata_str)
        except yaml.YAMLError:
            print(f"Error: Unable to parse YAML in {file_path}")
            continue

        # Extract content and source path
        content = parts[2].strip("\n")
        source_path = extract_path(file_path)

        # Split content based on headers
        headers_to_split_on = [
            ("#", "H1"),
            ("##", "H2"),
            ("###", "H3"),
            ("####", "H4"),
            ("#####", "H5"),
            ("######", "H6"),
        ]
        markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
        splits = markdown_splitter.split_text(content)

        # Create Document objects
        for split in splits:
            combined_text = f"Title: {extract_meta_dict['title']}\n"
            combined_text += f"Description: {extract_meta_dict['description']}\n"
            combined_text += f"Keywords: {extract_meta_dict['keywords']}\n"
            if split.metadata:
                for header_level in ["H1", "H2", "H3", "H4", "H5", "H6"]:
                    if header_level in split.metadata:
                        combined_text += f"Subtitle: {split.metadata[header_level]}\n"
                        break
            combined_text += f"Content: {split.page_content}\n"

            # Create the document with source_path in metadata
            doc = Document(
                page_content=combined_text,
                metadata={
                    "title": extract_meta_dict["title"],
                    "description": extract_meta_dict["description"],
                    "keywords": extract_meta_dict["keywords"],
                    "source_path": source_path,
                }
            )
            all_chunks.append(doc)
    return all_chunks

def create_embeddings_model(
    model_name = "text-embedding-3-large",
    chunk_size: int = 2048,
    api_key = os.environ["OPENAI_API_KEY"],
    api_version = "2024-02-01",
    azure_endpoint = "https://azure-preone-openai-sweden-central-01-chen.openai.azure.com/",
):
    # Create an AzureOpenAIEmbeddings model
    return AzureOpenAIEmbeddings(
        model=model_name,
        chunk_size=chunk_size,
        api_key=api_key,
        api_version=api_version,
        azure_endpoint=azure_endpoint,
    )


def create_vectorstore_batch(
        custom_chunks: List[Document],  
        store_db_nm: str = "test",
        model: str = "text-embedding-3-large",
        batch_size: int = 100,
        retry_delay: int = 60
):
    # Create a vectorstore from a list of Document objects
    embeddings = create_embeddings_model(chunk_size=2048)
    vectordb = None

    for i in range(0, len(custom_chunks), batch_size):
        batch = custom_chunks[i: i + batch_size]

        documents = [
            {
                "page_content": doc.page_content,
                "metadata": {
                    "title": doc.metadata.get("title", ""),
                    "description": doc.metadata.get("description", ""),
                    "keywords": doc.metadata.get("keywords", ""),
                    "source_path": doc.metadata.get("source_path", ""),
                    "chunk_id": i + j,
                }
            } for j, doc in enumerate(batch)
        ]

        while True:
            try:
                total_batches = (len(custom_chunks) + batch_size - 1) // batch_size
                current_batch = i // batch_size + 1
                print(f"Processing batch {current_batch}/{total_batches}")

                if vectordb is None:
                    vectordb = FAISS.from_texts(
                        [doc["page_content"] for doc in documents],
                        embeddings,
                        metadatas=[doc["metadata"] for doc in documents]
                    )
                else:
                    vectordb.add_texts(
                        [doc["page_content"] for doc in documents],
                        metadatas=[doc["metadata"] for doc in documents]
                    )
                break
            except Exception as e:
                print(f"Error processing batch: {e}")
                print(f"Retrying in {retry_delay} seconds...")
                time.sleep(retry_delay)
                retry_delay *= 2

        if vectordb:
            export_db_path = f"/home/Preda/user/Sony_InternProj/data/vectordb_{store_db_nm}"
            vectordb.save_local(export_db_path)
    return vectordb


def load_vectorstore(load_db_path):
    embeddings = create_embeddings_model()
    vectordb = FAISS.load_local(load_db_path, embeddings, allow_dangerous_deserialization=True)
    return vectordb

def search_vectorstore(
        vectorestore,
        query: str,
        k: int = 5
):
    return vectorestore.similarity_search_with_score(query, k=k)

def create_prompt(documents: List[tuple], query: str) -> str:
    prompt = f"質問に対して、以下の情報を踏まえ、具体的かつ詳細に回答してください：{query}\n\n"
    prompt += "参考情報：\n"
    for doc, _ in documents:
        prompt += f"- {doc.page_content}\n"
    
    prompt += "\n回答を作成する際の指示：\n"
    prompt += "1. 質問を繰り返さず、直接的な答えを提供してください。\n"
    prompt += "2. 関連する詳細や理由を含め、簡潔かつ論理的に回答してください。\n"
    prompt += "3. 回答は明確で、質問の重要なポイントを網羅してください。\n"
    prompt += "4. 情報不足を避けるため、答えを適度な長さに保ってください。\n"
    prompt += "5. 回答には無駄な表現やまとめの言葉を避け、質問に対する最適な情報を提供してください。\n"
    
    return prompt


def generate_answer(prompt: str) -> str:
    client = AzureOpenAI(
    azure_endpoint = "https://azure-preone-openai-sweden-central-01-chen.openai.azure.com/",
    api_key = os.environ["OPENAI_API_KEY"],
    api_version="2024-02-01"
    )
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "あなたは簡潔で直接的な回答を提供する専門家です。不要な説明や前置きを避け、質問に対して即座に核心を突いた回答をします。"},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content


In [3]:
root_dir = "/home/Preda/user/Sony_InternProj/content"
jp_md_files = find_jp_md_files(root_dir)
print("len of jp_md_files: ", len(jp_md_files))
# for file in jp_md_files:
#     print(file)
all_chunks = process_md_files(jp_md_files)
print("len of all_chunks: ", len(all_chunks))

len of jp_md_files:  577
len of all_chunks:  1068


In [4]:
all_chunks

[Document(metadata={'title': '各画面の操作と説明', 'description': '', 'keywords': '', 'source_path': 'operating_instruction'}, page_content='Title: 各画面の操作と説明\nDescription: \nKeywords: \nContent: Prediction Oneの各画面での操作について説明します。  \n<h3>設定・マニュアル</h3>\n<table class="select-screen">\n<tr>\n<td>\n<a href="manual/index.html">\n<img src="img/t_slide1.png">\n<p>チュートリアル画面</p>\n</a>\n</td>\n<td>\n<a href="setting/setting/index.html">\n<img src="img/t_slide10.png">\n<p>設定画面(デスクトップ版)</p>\n</a>\n</td>\n<td>\n<a href="setting/setting_cloud/index.html">\n<img src="img/t_slide10.png">\n<p>設定画面(クラウド版)</p>\n</a>\n</td>\n{{< dist_type_exclude "RDC" >}}\n<td>\n<a href="usage/index.html">\n<img src="img/t_slide73.png">\n<p>使用状況画面</p>\n</a>\n</td>\n{{< /dist_type_exclude >}}\n</tr>\n<tr>\n{{< dist_type_exclude "RDC" >}}\n<td>\n<a href="portal/index.html">\n<img src="img/t_slide75.png">\n<p>ユーザポータル画面（テナント管理画面）</p>\n</a>\n</td>\n<td>\n<a href="setting/licence/index.html">\n<img src="img/t_slide11.png">\n<p>ライセンス登録画面</

In [5]:
vector_db_path = f"/home/Preda/user/Sony_InternProj/data/vectordb_test"  # 你的路径

# 检查索引是否已经存在
if not os.path.exists(vector_db_path):
    print(f"Creating new vector database at {vector_db_path}")
    vector_db = create_vectorstore_batch(
        custom_chunks=all_chunks,
        store_db_nm="test", 
        batch_size=100,
    )
else:
    print(f"Loading existing vector database from {vector_db_path}")
    vector_db = load_vectorstore(vector_db_path)

Loading existing vector database from /home/Preda/user/Sony_InternProj/data/vectordb_test


In [23]:
query = "Prediction Oneのアルゴリズムやパラメータをカスタマイズして使用することは可能でしょうか？"
embedding_model = create_embeddings_model()
query_vector = embedding_model.embed_query(query)
len(query_vector)

3072

In [6]:
# query = "Prediction Oneのアルゴリズムやパラメータをカスタマイズして使用することは可能でしょうか？"
# results = search_vectorstore(vector_db, query, k=5)
# # print(results)
# prompt = create_prompt(results, query)
# answer = generate_answer(prompt)
# print("生成された回答: ", answer)

In [7]:
# results

In [8]:
# query = df["質問"][1]
# results = search_vectorstore(vector_db, query, k=5)
# results

In [14]:
file_path = "/home/Preda/user/Sony_InternProj/data/20240919/マニュアルQA(初級RAG)_with_scores のコピー.xlsx"
df = pd.read_excel(file_path)
df = df[:52]
# df["生成答え"] = df["質問"].apply(lambda x: generate_answer(create_prompt(search_vectorstore(vector_db, x, k=10), x)))

In [15]:
df["検索結果"] = df["質問"].apply(lambda x: search_vectorstore(vector_db, x, k=5))
df.head()

,QA番号,担当,質問カテゴリ,質問,答え（マニュアルのみ参照）,答え（マニュアル+一般知識）,マニュアル該当箇所,参考,生成答え,検索結果,source_path,DocumentCount,Precision@k,Recall@k,METEOR,faithfulness,answer_relevancy,context_precision,answer_correctness
0,NaN,山本,NaN,予測したい項目との関連度や相関係数とは何ですか？,関連度スコアとは、その項目と予測したい項目との間の関連度を示したものです。\n具体的には以下...,関連度スコアとは、その項目と予測したい項目との間の関連度を示したものです。\n具体的には以下...,terminology/corr,C&R FAQ 2\n相関係数についてマニュアルに記載はない,予測したい項目との関連度や相関係数は、その項目と予測対象となる項目間の関連性や関係性を数値化...,[(page_content='Title: 関連度スコア\nDescription: 関連...,"['terminology/corr', 'terminology/corr', 'term...",10.0,0.3,1.0,0.047619,0.40,0.450839,1.0,0.286911
1,NaN,山本,NaN,寄与度をプレゼンに使おうとしていて、寄与度のバーチャート(添付しています)を、エクセルで再現...,二値分類の場合、「顧客行動予測に基づいたターゲティングのチュートリアル」の例だと、赤や青のバ...,二値分類の場合、「顧客行動予測に基づいたターゲティングのチュートリアル」の例だと、赤や青のバ...,tips/new_features/contribution_csv,C&R FAQ 4,まず、CSVファイルをエクセルにインポートします。その後、以下の手順で赤と青の割合を再現しま...,[(page_content='Title: 予測寄与度のcsv出力\nDescriptio...,"['tips/new_features/contribution_csv', 'tips/n...",10.0,1.0,0.5,0.010846,NaN,0.554063,1.0,0.897431
2,NaN,山本,NaN,説明変数に文字データを選択する際、文字列orテキストで選ぶことができるかと思いますが、この2...,たとえばアンケートにおいて現在済んでいる県を選択する欄などの、 いくつかの限定された種類の文...,たとえばアンケートにおいて現在済んでいる県を選択する欄などの、 いくつかの限定された種類の文...,terminology/string\ntips/advice/advice_23101,C&R FAQ 9,文字データの扱いは以下のように使い分けると効果的です。\n\n**文字列型**:\n- **...,[(page_content='Title: 文字列の処理\nDescription: たと...,"['terminology/string', 'tips/advice/advice_231...",10.0,0.3,1.0,0.013699,0.45,0.450050,1.0,0.717601
3,NaN,山本,NaN,データセットとして画像ファイルを利用することは可能でしょうか？画像と数値の組み合わせを学習デ...,マニュアルにデータセットとして画像ファイルを利用することに関する記述はありません。,申し訳ありませんがPrediction Oneで利用できるデータセットは表形式データ(csv...,NaN,C&R FAQ 10,画像ファイルをデータセットとして利用することは可能です。しかし、Prediction One...,[(page_content='Title: 学習に使用する数値ベクトルを生成できませんでし...,"['trouble/message/e78', 'trouble/createmodel/c...",10.0,0.0,0.0,0.028902,NaN,0.563972,0.0,0.812533
4,NaN,山本,NaN,Prediction Oneにおいては、数値予測のモデルを作成したいとき、パラメータを選択す...,マニュアルに多重共線性に関する記述はありません。,数値予測をするにあたっては、多重共線性がある項目もそのまま使用していただいて問題ありません。...,NaN,C&R FAQ 12,はい、数値予測モデルを作成する際には多重共線性をケアする必要があります。多重共線性が存在する...,[(page_content='Title: 系列数を増やして同時に予測モデルの作成を行いま...,"['tips/advice/advice_20503', 'prediction_one/k...",10.0,0.0,0.0,0.070423,NaN,0.534453,0.0,0.157915


In [16]:
df.to_excel("/home/Preda/user/Sony_InternProj/data/20240919/マニュアルQA(初級RAG)_with_scores のコピー.xlsx", index=False)